<a href="https://colab.research.google.com/github/abalaji-blr/Session12_HandlingDataset/blob/main/Session12_Gen_HandlingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 12: Generators and Handling Dataset

## About Data

For this project you have 4 files containing information about persons.

The files are:
* `personal_info.csv` -   personal information such as name, gender, etc. (one row per person)
* `vehicles.csv` -   what vehicle people own (one row per person)
* `employment.csv` -   where a person is employed (one row per person)
* `update_status.csv` -   when the person's data was created and last updated

Each file contains a key, `SSN`, which **uniquely** identifies a person.

This key is present in **all** four files.

You are guaranteed that the same SSN value is present in **every** file, and that it only appears **once per file**.

In addition, the files are all sorted by SSN, i.e. the SSN values appear in the same order in each file.

## Objectives

## Objective

##### Goal 1

Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.

##### Goal 2

Create a single iterable that combines all the columns from all the iterators.

The iterable should yield named tuples containing all the columns.
Make sure that the SSN's across the files match!

All the files are guaranteed to be in SSN sort order, and every SSN is unique, and every SSN appears in every file.

Make sure the SSN is not repeated 4 times - one time per row is enough!

##### Goal 3

Next, you want to identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the `last_updated` field from the `status_update` file.

##### Goal 4

Find the largest group of car makes for each gender.

Possibly more than one such group per gender exists (equal sizes).

##### Hints

You will not be able to use a simple split approach here, as I explain in the video.

Instead you should use the `csv` module and the `reader` function.

Here's a simple example of how to use it - you will need to expand on this for your project goals, but this is a good starting point.

## Hints About Handling Files

In [ ]:
import csv

def read_file(file_name):
    with open(file_name) as f:
        rows = csv.reader(f, delimiter=',', quotechar='"')
        yield from rows

In [ ]:
from itertools import islice

# rows = read_file('personal_info.csv')
# for row in islice(rows, 5):
#     print(row)

As you can see, the data is already separated into a list containing the individual fields - but of course they are all just strings.

---

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
import sys
sys.path.append('/content/gdrive/MyDrive/Python/Assignment')

In [3]:
dir_loc = '/content/gdrive/MyDrive/Python/Assignment/'
per_file = 'personal_info.csv'
emp_file = 'employment.csv'
veh_file = 'vehicles.csv'
status_file = 'update_status.csv'

## Imports

In [4]:
import csv
from itertools import islice
from datetime import datetime
from functools import namedtuple
import itertools
import pprint # pretty print

### Examine File Content

In [5]:
def read_file(filename):
  with open(filename) as f:
    rows = csv.reader(f,delimiter =',', quotechar = '"')
    yield from rows

def examine_file_content(filename):
  rows = read_file(filename)

  # just examine first 5 rows.
  for row in islice(rows, 5):
    print(row)

In [6]:
examine_file_content(dir_loc+per_file)

['ssn', 'first_name', 'last_name', 'gender', 'language']
['100-53-9824', 'Sebastiano', 'Tester', 'Male', 'Icelandic']
['101-71-4702', 'Cayla', 'MacDonagh', 'Female', 'Lao']
['101-84-0356', 'Nomi', 'Lipprose', 'Female', 'Yiddish']
['104-22-0928', 'Justinian', 'Kunzelmann', 'Male', 'Dhivehi']


In [7]:
examine_file_content(dir_loc+emp_file)

['employer', 'department', 'employee_id', 'ssn']
['Stiedemann-Bailey', 'Research and Development', '29-0890771', '100-53-9824']
['Nicolas and Sons', 'Sales', '41-6841359', '101-71-4702']
['Connelly Group', 'Research and Development', '98-7952860', '101-84-0356']
['Upton LLC', 'Marketing', '56-9817552', '104-22-0928']


In [8]:
examine_file_content(dir_loc+veh_file)

['ssn', 'vehicle_make', 'vehicle_model', 'model_year']
['100-53-9824', 'Oldsmobile', 'Bravada', '1993']
['101-71-4702', 'Ford', 'Mustang', '1997']
['101-84-0356', 'GMC', 'Yukon', '2005']
['104-22-0928', 'Oldsmobile', 'Intrigue', '2000']


In [9]:
examine_file_content(dir_loc+status_file)

['ssn', 'last_updated', 'created']
['100-53-9824', '2017-10-07T00:14:42Z', '2016-01-24T21:19:30Z']
['101-71-4702', '2017-01-23T11:23:17Z', '2016-01-27T04:32:57Z']
['101-84-0356', '2017-10-04T11:21:30Z', '2016-09-21T23:04:07Z']
['104-22-0928', '2017-03-28T12:38:29Z', '2016-04-15T11:37:17Z']


### Goal 1 : Create Iterators For Each File.


Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.

#### Build Validators For Values

In [10]:
# build parser for each row value
def parse_int(value):
  try:
    return int(value)
  except ValueError:
    return None

def parse_string(value):
  try:
    return str(value)
  except ValueError:
    return None

def parse_date(value):
  try:
    date = datetime.strptime(value, '%Y-%m-%dT%H:%M:%SZ')
    #print(date)
    return(date)
  except ValueError:
    return None

# parse SSN: 
# input: 100-53-9824 
# output:100539824
def parse_ssn(value):
  try:
    val = value.replace('-', '')
    return int(val)
  except ValueError:
    return None

def parse_emp_id(value):
  try:
    val = value.replace('-', '')
    return int(val)
  except ValueError:
    return None

In [11]:
print(parse_int('4006478550'))
print(parse_string(123))
print(parse_ssn('100-53-9824'))
parse_date('2017-10-07T00:14:42Z')

4006478550
123
100539824


datetime.datetime(2017, 10, 7, 0, 14, 42)

#### Incorporate Validators to Files and Build Generic Generator

In [12]:
def read_file_gen(filename, row_parser, nt_name = 'DATA'):
  '''
  Inputs: filename, row value parser
  Outputs: Namedtuple
  '''
  with open(filename) as f:
    rows = csv.reader(f,delimiter =',', quotechar = '"')

    hdr = next(rows)
    #build namedtuple
    nt_name = namedtuple(nt_name, hdr)

    for r in rows:
      row_value = [func(val) for func, val in zip(row_parser, r)]
      yield nt_name._make(row_value)

##### Personal File Generator

In [13]:
 #define the row value parsers
row_parser = (parse_ssn,
                  parse_string,
                  parse_string,
                  parse_string,
                  parse_string)
  
gen = read_file_gen(dir_loc+ per_file, row_parser, 'PER_DATA')

for row in islice(gen, 5):
  print(row)

PER_DATA(ssn=100539824, first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
PER_DATA(ssn=101714702, first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
PER_DATA(ssn=101840356, first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')
PER_DATA(ssn=104220928, first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi')
PER_DATA(ssn=104847144, first_name='Claudianus', last_name='Brixey', gender='Male', language='Afrikaans')


##### Employment file generator

In [14]:
#['employer', 'department', 'employee_id', 'ssn']
#['Stiedemann-Bailey', 'Research and Development', '29-0890771', '100-53-9824']
emp_row_parser = (parse_string,
                  parse_string,
                  parse_emp_id,
                  parse_ssn)

emp_gen = read_file_gen(dir_loc+emp_file, emp_row_parser, 'EMP_DATA')

for row in islice(emp_gen, 5):
  print(row)

EMP_DATA(employer='Stiedemann-Bailey', department='Research and Development', employee_id=290890771, ssn=100539824)
EMP_DATA(employer='Nicolas and Sons', department='Sales', employee_id=416841359, ssn=101714702)
EMP_DATA(employer='Connelly Group', department='Research and Development', employee_id=987952860, ssn=101840356)
EMP_DATA(employer='Upton LLC', department='Marketing', employee_id=569817552, ssn=104220928)
EMP_DATA(employer='Zemlak-Olson', department='Business Development', employee_id=462886707, ssn=104847144)


##### Vechile File Generator

In [15]:
#['ssn', 'vehicle_make', 'vehicle_model', 'model_year']
#['100-53-9824', 'Oldsmobile', 'Bravada', '1993']

veh_row_parser = (parse_ssn,
                  parse_string,
                  parse_string,
                  parse_int)

veh_file_gen = read_file_gen(dir_loc+veh_file, veh_row_parser, 'VEH_DATA')

for row in islice(veh_file_gen, 5):
  print(row)

VEH_DATA(ssn=100539824, vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
VEH_DATA(ssn=101714702, vehicle_make='Ford', vehicle_model='Mustang', model_year=1997)
VEH_DATA(ssn=101840356, vehicle_make='GMC', vehicle_model='Yukon', model_year=2005)
VEH_DATA(ssn=104220928, vehicle_make='Oldsmobile', vehicle_model='Intrigue', model_year=2000)
VEH_DATA(ssn=104847144, vehicle_make='Ford', vehicle_model='Crown Victoria', model_year=2008)


##### Status File Generator

In [16]:
#['ssn', 'last_updated', 'created']
#['100-53-9824', '2017-10-07T00:14:42Z', '2016-01-24T21:19:30Z']

stat_row_parser = (parse_ssn,
                   parse_date,
                   parse_date)

stat_file_gen = read_file_gen(dir_loc+status_file, stat_row_parser, 'STAT_DATA')

for row in islice(stat_file_gen, 5):
  print(row)

STAT_DATA(ssn=100539824, last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30))
STAT_DATA(ssn=101714702, last_updated=datetime.datetime(2017, 1, 23, 11, 23, 17), created=datetime.datetime(2016, 1, 27, 4, 32, 57))
STAT_DATA(ssn=101840356, last_updated=datetime.datetime(2017, 10, 4, 11, 21, 30), created=datetime.datetime(2016, 9, 21, 23, 4, 7))
STAT_DATA(ssn=104220928, last_updated=datetime.datetime(2017, 3, 28, 12, 38, 29), created=datetime.datetime(2016, 4, 15, 11, 37, 17))
STAT_DATA(ssn=104847144, last_updated=datetime.datetime(2018, 2, 19, 1, 34, 33), created=datetime.datetime(2016, 3, 15, 14, 7, 57))


## Goal 2 : Create a single iterable for all files.

Create a single iterable that combines all the columns from all the iterators.

The iterable should yield named tuples containing all the columns.
Make sure that the SSN's across the files match!

In [18]:
def read_file_gen_return_tuple(filename, row_parser):
  '''
  inputs: file-name, row_parser
  output: list of tuples, [(col-name, val), (col-name2, val2) ... ]
  '''
  with open(filename) as f:
    rows = csv.reader(f,delimiter =',', quotechar = '"')
    hdr = next(rows)
    
    for r in rows:
      row_value = [func(val) for func, val in zip(row_parser, r)]
      yield list(zip(hdr, row_value))

#### Get All Four Generators and Merge Based on SSN

In [19]:
gen1 = read_file_gen_return_tuple(dir_loc+ per_file, row_parser)
gen2 = read_file_gen_return_tuple(dir_loc+emp_file, emp_row_parser)
gen3 = read_file_gen_return_tuple(dir_loc+veh_file, veh_row_parser)
gen4 = read_file_gen_return_tuple(dir_loc+status_file, stat_row_parser)

for row in islice(zip(gen1, gen2, gen3, gen4), 5):
  # get the flatten list
  flatten_list = list(itertools.chain(*row))

  # need to remove the multiple definition of same key.
  # try using set to remove redundant items
  s = set(flatten_list)
  print(s)

{('vehicle_model', 'Bravada'), ('employer', 'Stiedemann-Bailey'), ('employee_id', 290890771), ('model_year', 1993), ('last_updated', datetime.datetime(2017, 10, 7, 0, 14, 42)), ('created', datetime.datetime(2016, 1, 24, 21, 19, 30)), ('vehicle_make', 'Oldsmobile'), ('last_name', 'Tester'), ('gender', 'Male'), ('first_name', 'Sebastiano'), ('ssn', 100539824), ('department', 'Research and Development'), ('language', 'Icelandic')}
{('created', datetime.datetime(2016, 1, 27, 4, 32, 57)), ('department', 'Sales'), ('language', 'Lao'), ('vehicle_model', 'Mustang'), ('last_name', 'MacDonagh'), ('model_year', 1997), ('vehicle_make', 'Ford'), ('last_updated', datetime.datetime(2017, 1, 23, 11, 23, 17)), ('employer', 'Nicolas and Sons'), ('first_name', 'Cayla'), ('ssn', 101714702), ('gender', 'Female'), ('employee_id', 416841359)}
{('vehicle_model', 'Yukon'), ('employer', 'Connelly Group'), ('employee_id', 987952860), ('vehicle_make', 'GMC'), ('created', datetime.datetime(2016, 9, 21, 23, 4, 7)),

#### Create Single Generator

In [20]:
def get_name_value(complete_row):
  # flatten the list
  # need to remove the multiple definition of same key.
  # try using set
  flatten_list = set(complete_row)

  hdr = [name[0] for name in flatten_list]
  val = [name[1] for name in flatten_list]
  return(hdr, val)
  
def create_single_gen(gen1, gen2, gen3, gen4):
  '''
  Build single generator.
  '''
  # first create named tuple
  for row in islice(zip(gen1, gen2, gen3, gen4), 1):
    # create the namedtuple
    complete_row = list(itertools.chain(*row))
    hdr, val = get_name_value(complete_row)
    CombinedData = namedtuple('CombinedData', hdr)
    #print(flatten_list)
    yield CombinedData._make(val)
    
  # yield rest of the records
  for row in zip(gen1, gen2, gen3, gen4):
    complete_row = list(itertools.chain(*row))
    hdr, val = get_name_value(complete_row)
    di = dict()
    for key, val in zip(hdr, val):
      di[key] = val
    #print(di)
    yield CombinedData(**di)

In [21]:
single_gen = create_single_gen(gen1, gen2, gen3, gen4)

for row in islice(single_gen, 5):
  print(row)

CombinedData(employer='Kohler, Bradtke and Davis', last_name='Aggett', created=datetime.datetime(2016, 7, 23, 17, 58, 35), ssn=105275541, first_name='Federico', employee_id=800975518, model_year=2001, vehicle_model='Mustang', vehicle_make='Ford', language='Chinese', gender='Male', department='Support', last_updated=datetime.datetime(2017, 7, 24, 8, 58, 52))
CombinedData(employer='Roberts, Torphy and Dach', last_name='McAvey', created=datetime.datetime(2016, 12, 15, 5, 46, 43), ssn=105857486, first_name='Angelina', employee_id=774895332, model_year=2008, vehicle_model='300', vehicle_make='Chrysler', language='Punjabi', gender='Female', department='Human Resources', last_updated=datetime.datetime(2018, 2, 14, 11, 32, 39))
CombinedData(employer='Lind-Jast', last_name='Apfel', created=datetime.datetime(2016, 3, 24, 3, 43, 3), ssn=105915022, first_name='Moselle', employee_id=796418731, model_year=2000, vehicle_model='Hombre Space', vehicle_make='Isuzu', language='Latvian', gender='Female', 

## Goal 3: Create Iterator To Ignore Stale Records

Next, you want to identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the `last_updated` field from the `status_update` file.

In [22]:
def is_record_stale(complete_row):
  '''
  The record is stale if not updated on or after
  the reference date (Mar/1/2017)
  '''
  # yyyy/mm/dd
  reference_date = datetime(2017, 3, 1).date()
  for item in complete_row:
    key = item[0]
    value = item[1]
    if key == 'last_updated':
      #print(value.date())
      if value.date() < reference_date:
        return True
      else:
        return False

def create_single_gen(gen1, gen2, gen3, gen4):
  '''
  Create Iterator to ignore stale records
  '''
  # first create named tuple
  for row in islice(zip(gen1, gen2, gen3, gen4), 1):
    # create the namedtuple
    complete_row = list(itertools.chain(*row))
    hdr, val = get_name_value(complete_row)
    CombinedData = namedtuple('CombinedData', hdr)
    yield CombinedData._make(val)
    
  # yield rest of the records
  for row in zip(gen1, gen2, gen3, gen4):
    complete_row = list(itertools.chain(*row))

    if is_record_stale(complete_row):
      continue

    # build dictionary for namedtuple
    hdr, val = get_name_value(complete_row)
    di = dict()
    for key, val in zip(hdr, val):
      di[key] = val
      
    yield CombinedData(**di)

In [23]:
single_gen = create_single_gen(gen1, gen2, gen3, gen4)
for row in islice(single_gen, 5):
  print(row)

CombinedData(department='Accounting', employee_id=315735282, vehicle_make='Volkswagen', vehicle_model='Touareg', created=datetime.datetime(2016, 5, 16, 21, 21, 36), first_name='Martino', ssn=106363293, last_updated=datetime.datetime(2017, 3, 18, 18, 24, 17), gender='Male', language='Tok Pisin', employer='Leffler-Hahn', last_name='Tregoning', model_year=2008)
CombinedData(department='Marketing', employee_id=339146042, vehicle_make='Ford', vehicle_model='Mustang', created=datetime.datetime(2016, 9, 12, 22, 50, 5), first_name='Amberly', ssn=110843641, last_updated=datetime.datetime(2017, 9, 4, 19, 2, 3), gender='Female', language='Papiamento', employer='Lueilwitz LLC', last_name='Huws', model_year=1990)
CombinedData(department='Accounting', employee_id=390400385, vehicle_make='Ford', vehicle_model='Fiesta', created=datetime.datetime(2016, 8, 21, 7, 36, 17), first_name='Giacopo', ssn=111351034, last_updated=datetime.datetime(2017, 3, 18, 14, 51, 4), gender='Male', language='Gagauz', employ

## Goal 4: Find the largest group of car makes for each gender.

Possibly more than one such group per gender exists (equal sizes).

In [28]:
# get the generators
gen1 = read_file_gen_return_tuple(dir_loc+ per_file, row_parser)
gen2 = read_file_gen_return_tuple(dir_loc+emp_file, emp_row_parser)
gen3 = read_file_gen_return_tuple(dir_loc+veh_file, veh_row_parser)
gen4 = read_file_gen_return_tuple(dir_loc+status_file, stat_row_parser)

# create single iteraor
single_gen = create_single_gen(gen1, gen2, gen3, gen4)


def add_to_dict(dt, key):
  if key in dt:
    dt[key] += 1
  else:
    dt[key] = 1

male_veh_dict = dict()
female_veh_dict = dict()

for row in single_gen:
  nt = row
  if nt.gender.lower() == 'male':
    add_to_dict(male_veh_dict, nt.vehicle_make)
  else:
    add_to_dict(female_veh_dict, nt.vehicle_make)

In [29]:
print('------Vehicle Make of Females----------')
pprint.pprint(female_veh_dict)
print(f'Total number of vehicles {sum(female_veh_dict.values())}')

print('-------Vehicle Make of Males----------')
pprint.pprint(male_veh_dict)
print(f'Total number of vehicles {sum(male_veh_dict.values())}')

------Vehicle Make of Females----------
{'Acura': 9,
 'Aston Martin': 2,
 'Audi': 13,
 'Austin': 1,
 'BMW': 12,
 'Bentley': 4,
 'Bugatti': 1,
 'Buick': 11,
 'Cadillac': 6,
 'Chevrolet': 42,
 'Chrysler': 6,
 'Dodge': 17,
 'Eagle': 1,
 'Ford': 42,
 'GMC': 22,
 'Geo': 1,
 'Honda': 8,
 'Hyundai': 4,
 'Infiniti': 9,
 'Isuzu': 3,
 'Jaguar': 3,
 'Jeep': 5,
 'Kia': 9,
 'Lamborghini': 2,
 'Land Rover': 8,
 'Lexus': 15,
 'Lincoln': 4,
 'Lotus': 5,
 'Mazda': 13,
 'Mercedes-Benz': 17,
 'Mercury': 5,
 'Mitsubishi': 22,
 'Morgan': 1,
 'Nissan': 12,
 'Oldsmobile': 8,
 'Panoz': 1,
 'Plymouth': 3,
 'Pontiac': 14,
 'Porsche': 3,
 'Rolls-Royce': 1,
 'Saab': 3,
 'Saturn': 3,
 'Scion': 2,
 'Subaru': 6,
 'Suzuki': 12,
 'Toyota': 20,
 'Volkswagen': 10,
 'Volvo': 13}
Total number of vehicles 434
-------Vehicle Make of Males----------
{'Acura': 7,
 'Aptera': 1,
 'Aston Martin': 3,
 'Audi': 14,
 'Austin': 1,
 'BMW': 12,
 'Bentley': 3,
 'Buick': 13,
 'Cadillac': 9,
 'Chevrolet': 30,
 'Chrysler': 3,
 'Corbin': 1,